In [1]:
import pandas as pd
path = r'../modelo_contrato_joao_csv.csv'

df = pd.read_csv(path, sep=';', encoding='latin1')
print(df.columns)
df.head(20)

Index(['RAZAO_SOCIAL', 'CNPJ', 'ESTADO', 'CIDADE', 'ENDERECO', 'BAIRRO',
       'COMPLEMENTO', 'CEP', 'DATA_CONTRATACAO', 'VALOR', 'COD_BANCO',
       'NUM_CONTA', 'AGENCIA', 'NOME_BANCO', 'CNAE'],
      dtype='object')


,RAZAO_SOCIAL,CNPJ,ESTADO,CIDADE,ENDERECO,BAIRRO,COMPLEMENTO,CEP,DATA_CONTRATACAO,VALOR,COD_BANCO,NUM_CONTA,AGENCIA,NOME_BANCO,CNAE
0,DAVI RICARDO DA SILVA CONSULTORIA EM TECNOLOGI...,64.251.684/0001-87,Ceará,Caucaia,"Rua Galiente, n° 222",Parque Guadalajara(Jurema),-,61.650-190,05/01/2026,"R$ 10.000,00",Cod 301,31110380-6,agência 0001,Contabilizei Bank,65.04-0-00 - Consultoria em tecnologia da info...
1,ISNARD SANTOS NETO TECNOLOGIA DA INFORMAÇÃO,64.191.236/0001-35,Ceará,Fortaleza,"Rua Doutor Alfredo Wayne, n° 55",Bairro de Fátima,AP 502 BL B,60.415-065,05/01/2026,"R$ 10.000,00",Cod 336,040842159-2,agência 0001,C6 BANK,62.04-0-00 - Consultoria em tecnologia da info...
2,ALLANA.DEV LTDA,64.344.269/0001-78,Ceará,Fortaleza,"Rua Valdenia Soares, nº 236",Granja lisboa,-,60.540-355,12/01/2026,"R$ 16.000,00",Cod 260,733929056-5,agência 0001,Nu Pagamentos S.A. - Instituição de pagamento,62.01-5-01 - Desenvolvimento de programas de c...
3,DVA TECNOLOGIA,44.756.074/0001-28,Ceará,Eusébio,Rua Francisca Maria da S.Barros,Encantada,-,61.772-244,12/01/2026,"R$ 16.000,00",Cod 260,79175803-1,agência 0001,Nu Pagamentos S.A. - Instituição de pagamento,"63.19-4-00 - Portais, provedores de conteúdo e..."
4,RENATA MIKAELY SANTIAGO MARQUES LTD,64.348.067/0001-02,Ceará,Fortaleza,Geraldo Barbosa,Bom Jardim,-,60.543-335,12/01/2026,"R$ 12.000,00",Cod 260,917446492-2,agência 0001,Nu Pagamentos S.A,62.04-0-00 - Consultoria em tecnologia da info...


In [2]:
import pandas as pd
from docx import Document
from num2words import num2words
import locale
from datetime import datetime

locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

def data_por_extenso(data_str):
    # Converte a string em um objeto datetime
    data = datetime.strptime(data_str, "%d/%m/%Y")
    
    # Lista com os nomes dos meses
    meses = [
        "janeiro", "fevereiro", "março", "abril", "maio", "junho",
        "julho", "agosto", "setembro", "outubro", "novembro", "dezembro"
    ]
    
    # Monta a data por extenso
    return f"{data.day} de {meses[data.month - 1]} de {data.year}"

def value_money(valor):
    locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
    valor_formatado = locale.format_string('%.2f', valor, grouping=True)
    return valor_formatado

def value_descrition(serie):
    valor_serie = serie
    valores_float = [float(v.replace("R$ ", "").replace(".", "").replace(",", ".")) for v in serie]
    serie = [num2words(v, lang='pt_BR', to='currency') for v in valores_float]
    return '{valor_serie} ({valor_extenso})'.format(valor_serie=valor_serie[0], valor_extenso=serie[0])

path = r'../Arquivos/MP_CONTRATO_TI _JUR_01.2026 RDSP JOAO CAMPOS.docx'

document = Document(path)

dict_modification = {
    '@razao_social@': 'Empresa Exemplo LTDA', # ok
    '@cnpj@': '12.345.678/0001-99 exemplo', # ok
    '@estado@': 'Fortaleza', # ok
    'XENDERECOX': 'Rua Exemplo, 123', # ok
    'XCOMPLEMENTOX': 'Apto 456', # ok
    'XCIDADEX': 'Fortaleza Exemplo', # ok
    'XBAIRROX': 'Centro',  # ok
    '@cep@': '60000-000', # ok
    'XDATA_DE_CONTRATACAOX': '23/01/2026', # ok
    'XVALOR_POR_EXTENSOX': 'R$ 30.000,00', # ok
    'XCOD_BANCO@' : '001', # ok
    'Xnum_contaX' : '1234567890', # ok
    'XAGENCIAX' : '1234-5', # ok
    'XNOME_BANCOX' : 'Banco Exemplo', # ok
    'XCNAEX': 'OO1- cNAE exemplo', # ok
}

dict_modification['XDATA_DE_CONTRATACAOX'] = data_por_extenso(dict_modification['XDATA_DE_CONTRATACAOX'])

print(50*"-")
valor = dict_modification['XVALOR_POR_EXTENSOX']
print(type(valor))
print(valor)
valor_extenso = value_descrition([valor])
print(valor_extenso)
print(50*"-")

dict_modification['XVALOR_POR_EXTENSOX'] = value_descrition([dict_modification['XVALOR_POR_EXTENSOX']])

list_modification = []

doc = Document(path)

for para in doc.paragraphs:
    for run in para.runs:
        for key, value in dict_modification.items():
            if key in run.text:
                run.text = run.text.replace(key, value)
                lista_de_substituicao = f"Substituído '{key}' por '{value}'"
                list_modification.append(lista_de_substituicao)
    print(para.text)

# mostrando os parágrafos do documento
# for para in document.paragraphs:
#     print(para.text)

--------------------------------------------------
<class 'str'>
R$ 30.000,00
R$ 30.000,00 (trinta mil reais)
--------------------------------------------------
CONTRATO DE PRESTAÇÃO DE SERVIÇOS EM TECNOLOGIA DA INFORMAÇÃO
HAPVIDA ASSISTÊNCIA MÉDICA S.A., doravante designada “CONTRATANTE”, com sede na Capital do Estado do Ceará, na Avenida Heráclito Graça, 406, Centro, Fortaleza, CEP 60140-061, inscrita no Cadastro Nacional de Pessoas Jurídicas do Ministério da Fazenda (CNPJ/MF) sob n.º 63.554.067/0001-98, neste ato, representada na forma de seus atos constitutivos,
e de outro lado, Empresa Exemplo LTDA, com sede na Capital do Estado de Fortaleza no Rua Exemplo, 123, Apto 456, Centro, Fortaleza Exemplo, 60000-000, inscrita no CNPJ/MF sob o n.º 12.345.678/0001-99 exemplo, neste ato, representada na forma de seus atos constitutivos, doravante denominada “CONTRATADA”, têm entre si, justo e contratado firmar o presente CONTRATO de prestação de serviços em tecnologia da informação (“CONTRAT

In [2]:
# criando um novo documento com as modificações
import pandas as pd
import os
from docx import Document
from num2words import num2words
import locale
from datetime import datetime
import numpy as np

locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

# Função para converter data para extenso
def data_por_extenso(data_str):
    # Converte a string em um objeto datetime
    data = datetime.strptime(data_str, "%d/%m/%Y")
    
    # Lista com os nomes dos meses
    meses = [
        "janeiro", "fevereiro", "março", "abril", "maio", "junho",
        "julho", "agosto", "setembro", "outubro", "novembro", "dezembro"
    ]
    
    # Monta a data por extenso
    return f"{data.day} de {meses[data.month - 1]} de {data.year}"

# Função para formatar valor monetário
def value_money(valor):
    locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
    valor_formatado = locale.format_string('%.2f', valor, grouping=True)
    return valor_formatado

# Função para converter valor monetário para extenso
def value_descrition(serie):
    valor_serie = serie
    valores_float = [float(v.replace("R$ ", "").replace(".", "").replace(",", ".")) for v in serie]
    serie = [num2words(v, lang='pt_BR', to='currency') for v in valores_float]
    return '{valor_serie} ({valor_extenso})'.format(valor_serie=valor_serie[0], valor_extenso=serie[0])

# definindo os caminhos dos arquivos
path_contrato = r'../Arquivos/MP_CONTRATO_TI _JUR_01.2026 RDSP JOAO CAMPOS.docx'
path_csv = r'../modelo_contrato_joao_csv.csv'
path_save = r'../Contratos_Gerados'

# lendo o arquivo CSV
df = pd.read_csv(path_csv, sep=';', encoding='latin1')

for num, dataframe in enumerate(df.itertuples(index=False)):
    dict_modification = {
        '@razao_social@': dataframe.RAZAO_SOCIAL,
        '@cnpj@': dataframe.CNPJ,
        '@estado@': dataframe.ESTADO,
        'XENDERECOX': dataframe.ENDERECO,
        'XCOMPLEMENTOX': dataframe.COMPLEMENTO,
        'XCIDADEX': dataframe.CIDADE,
        'XBAIRROX': dataframe.BAIRRO,
        '@cep@': dataframe.CEP,
        'XDATA_DE_CONTRATACAOX': dataframe.DATA_CONTRATACAO,
        'XVALOR_POR_EXTENSOX': dataframe.VALOR,
        'XCOD_BANCO@' : dataframe.COD_BANCO,
        'Xnum_contaX' : dataframe.NUM_CONTA,
        'XAGENCIAX' : dataframe.AGENCIA,
        'XNOME_BANCOX' : dataframe.NOME_BANCO,
        'XCNAEX': dataframe.CNAE,
    }

    dict_modification['XDATA_DE_CONTRATACAOX'] = data_por_extenso(dict_modification['XDATA_DE_CONTRATACAOX'])

    print(50*"-")
    valor = dict_modification['XVALOR_POR_EXTENSOX']
    print(type(valor))
    print(valor)
    valor_extenso = value_descrition([valor])
    print(valor_extenso)
    print(50*"-")
    valor_num_conta = dict_modification['XCOMPLEMENTOX']
    print('Valor do complemento:')
    print(type(valor_num_conta))
    print(valor_num_conta)
    # trocando os valores no dicionário de "nan" para string "-"
    if isinstance(valor_num_conta, float) and np.isnan(valor_num_conta):
        dict_modification['XCOMPLEMENTOX'] = '-'
    else:
        dict_modification['XCOMPLEMENTOX'] = valor_num_conta

    dict_modification['XVALOR_POR_EXTENSOX'] = value_descrition([dict_modification['XVALOR_POR_EXTENSOX']])

    list_modification = []

    doc = Document(path_contrato)

    for para in doc.paragraphs:
        for run in para.runs:
            for key, value in dict_modification.items():
                if key in run.text:
                    run.text = run.text.replace(key, str(value))
                    lista_de_substituicao = f"Substituído '{key}' por '{value}'"
                    list_modification.append(lista_de_substituicao)
        #print(para.text)
    nome_arquivo_salvo = f"{dataframe.RAZAO_SOCIAL.replace(' ', '_')}.docx"
    save_path = os.path.join(path_save, nome_arquivo_salvo)
    doc.save(save_path)
    print(f"{num+1} Arquivo salvo como: {nome_arquivo_salvo}")

--------------------------------------------------
<class 'str'>
 R$ 10.000,00 
 R$ 10.000,00  (dez mil reais)
--------------------------------------------------
Valor do complemento:
<class 'str'>
-
1 Arquivo salvo como: DAVI_RICARDO_DA_SILVA_CONSULTORIA_EM_TECNOLOGIA_DA_INFORMACAO_LTDA.docx
--------------------------------------------------
<class 'str'>
 R$ 10.000,00 
 R$ 10.000,00  (dez mil reais)
--------------------------------------------------
Valor do complemento:
<class 'str'>
AP 502 BL B
2 Arquivo salvo como: ISNARD_SANTOS_NETO_TECNOLOGIA_DA_INFORMAÇÃO.docx
--------------------------------------------------
<class 'str'>
 R$ 16.000,00 
 R$ 16.000,00  (dezesseis mil reais)
--------------------------------------------------
Valor do complemento:
<class 'str'>
-
3 Arquivo salvo como: ALLANA.DEV_LTDA.docx
--------------------------------------------------
<class 'str'>
 R$ 16.000,00 
 R$ 16.000,00  (dezesseis mil reais)
--------------------------------------------------
Valor do